In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from datetime import datetime
import json
import numpy as np
import os
import pandas as pd
import sys 
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.callbacks import History 
history = History()

Mounted at /content/drive


### Specify here the NN topology and the dataframe to be loaded

In [ ]:
#specify the NN topology here:
#input_layer_nodes = number_of_attributes
number_of_attributes = 24 
hidden_layer_nodes = 576
output_layer_nodes = 24
BATCH_SIZE = 10
EPOCHS = 10
#specify the dataset here:
#choose the dataframe which the NN will be trained and mo
df_number = 6
n_samples_train = 990000
n_samples_test = 10000


#attributes_array = ['proto', 'pktTotalCount', 'octetTotalCount', 'min_ps', 'max_ps', 'avg_ps', 'std_dev_ps', 'flowStart', 'flowEnd', 'flowDuration', 'min_piat', 'max_piat', 'avg_piat', 'std_dev_piat', 'f_pktTotalCount', 'f_octetTotalCount', 'f_min_ps', 'f_max_ps', 'f_avg_ps', 'f_std_dev_ps', 'f_flowStart', 'f_flowEnd', 'f_flowDuration', 'f_min_piat', 'f_max_piat', 'f_avg_piat', 'f_std_dev_piat', 'b_pktTotalCount', 'b_octetTotalCount', 'b_min_ps', 'b_max_ps', 'b_avg_ps', 'b_std_dev_ps', 'b_flowStart', 'b_flowEnd', 'b_flowDuration', 'b_min_piat', 'b_max_piat', 'b_avg_piat', 'b_std_dev_piat', 'flowEndReason']
attributes_array = ['proto', 'f_pktTotalCount', 'f_octetTotalCount', 'f_min_ps', 'f_max_ps', 'f_avg_ps', 'f_flowStart', 'f_flowEnd', 'f_flowDuration', 'f_min_piat', 'f_max_piat', 'f_avg_piat', 'b_pktTotalCount', 'b_octetTotalCount', 'b_min_ps', 'b_max_ps', 'b_avg_ps', 'b_flowStart', 'b_flowEnd', 'b_flowDuration', 'b_min_piat', 'b_max_piat', 'b_avg_piat', 'flowEndReason']
#attributes_array = ['82', '83', '81', '89', '15', '22', '84', '90', '88', '79', '87', '80', '17', '64', '60', '10', '29', '59', '63', '24', '62', '61', '91', '48', '47', '32', '34', '31', '33', '45', '46', '36', '37', '38', '92']    

###Define the model

In [ ]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(hidden_layer_nodes, activation='relu'), 
    tf.keras.layers.Dense(output_layer_nodes)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

###Import the data

In [ ]:
#Load dataframes
df_train = pd.read_csv(f"/content/drive/MyDrive/NN-P4/csv-files/df_6_train_24_attr_990000_samples.csv")
df_test = pd.read_csv(f"/content/drive/MyDrive/NN-P4/csv-files/df_6_test_24_attr_10000_samples.csv")
#Load labels
target_train = pd.read_csv('/content/drive/MyDrive/NN-P4/csv-files/df_6_train_target_990000_samples.csv')
target_test = pd.read_csv('/content/drive/MyDrive/NN-P4/csv-files/df_6_test_target_10000_samples.csv')
display(df_test)

,proto,f_pktTotalCount,f_octetTotalCount,f_min_ps,f_max_ps,f_avg_ps,f_flowStart,f_flowEnd,f_flowDuration,f_min_piat,...,b_min_ps,b_max_ps,b_avg_ps,b_flowStart,b_flowEnd,b_flowDuration,b_min_piat,b_max_piat,b_avg_piat,flowEndReason
0,17,10,563,48,131,56,1555964273,1555964836358,563,3,...,0,0,0,0,0,0,0,0,0,2
1,6,10,6167,40,2960,616,1556053574,1556053636477,61,0,...,40,4348,1056,1556053574,1556053636,1554497582902,0,60,4,3
2,6,4,367,40,235,91,1555955309,1555955309156,0,0,...,40,1616,569,1555955309,1555955309,1554399353845,0,0,0,2
3,6,14,2935,40,824,209,1556047747,1556047748715,1,0,...,40,2285,280,1556047747,1556047748,1554491700967,0,0,0,3
4,6,16,3417,40,906,213,1556029424,1556029664331,240,0,...,40,464,106,1556029424,1556029664,1554473634906,0,239,13,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,6,1,40,40,40,40,1556124962,1556124962,0,0,...,0,0,0,0,0,0,0,0,0,2
9996,6,7,1014,40,557,144,1556054273,1556054274031,0,0,...,40,2876,714,1556054273,1556054273,1554498219247,0,0,0,4
9997,6,70,13992,40,1570,199,1555969058,1555969315453,256,0,...,40,1509,387,1555969058,1555969315,1554413346394,0,45,2,3
9998,6,31,2674,52,588,86,1555967356,1555967641164,285,0,...,52,2888,1267,1555967356,1555967641,1554411673807,0,239,10,3


### NN training
 model weights file generation

 model parameters json file generation

In [ ]:
#necessário para corrigir erro, converte valores int não suportados para o tipo que o tensorflow espera.
target_train = np.asarray(target_train).astype(np.float32)
target_test = np.asarray(target_test).astype(np.float32)
df_train = np.asarray(df_train).astype(np.float32)
df_test = np.asarray(df_test).astype(np.float32)
# Begin NN training 
# Converts pandas dataframe to tensorflow object
df_train = tf.convert_to_tensor(df_train)
# Normalize the data
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(df_train)

# Execute Trainning 
print('Starting training for',number_of_attributes,'attributes')
model = get_basic_model()
model.fit(df_train, target_train, epochs=EPOCHS, verbose=1, batch_size=BATCH_SIZE)

# Calling `save('my_model')` creates a SavedModel folder `my_model`.
# model weights file generation
title_model_save = f'/content/drive/MyDrive/NN-P4/tf-models/model_with_{number_of_attributes}_attributes_{hidden_layer_nodes}_hidden_nodes.csv'
model.save(title_model_save) 

test_loss, one_test_acc =  model.evaluate(df_test,  target_test, verbose=1, batch_size=BATCH_SIZE)      
tf_predictions_probabilities = model.predict(df_test)
tf_predictions = []

for i,x in enumerate(tf_predictions_probabilities):
  #print("i:",i,"x_max:",x.max(),"x:",x)  
  j_max = x.argmax()
  tf_predictions.append(j_max)  

conf_m = tf.math.confusion_matrix(target_test,tf_predictions)
print(conf_m)

# Date for report file
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y %H:%M:%S")
# Report file content
report = {
    "datetime": dt_string,
    "attributes": attributes_array[0:number_of_attributes],
    "number_of_attributes": number_of_attributes,
    "hidden_layer_nodes": hidden_layer_nodes,
    "output_layer_nodes": output_layer_nodes,
    "accuracy_test": one_test_acc,
    "attributes_array": attributes_array,
    "batch_size": BATCH_SIZE,
    "epochs": EPOCHS

}
# Writes into the file
# model parameters json file generation
title = "/content/drive/MyDrive/NN-P4/nn-reports/nn_model_parameters_%s"%dt_string + "_with_%d_attributes.json"%number_of_attributes
with open(title, "w") as f:
  json.dump(report, f)

title_parameters_save = "/content/drive/MyDrive/NN-P4/nn-reports/nn_model_parameters.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)







Starting training for 24 attributes
Epoch 1/10
99000/99000 [==============================] - 219s 2ms/step - loss: 0.8489 - accuracy: 0.7414
Epoch 2/10
99000/99000 [==============================] - 207s 2ms/step - loss: 0.7853 - accuracy: 0.7656
Epoch 3/10
99000/99000 [==============================] - 201s 2ms/step - loss: 0.7662 - accuracy: 0.7741
Epoch 4/10
99000/99000 [==============================] - 205s 2ms/step - loss: 0.7682 - accuracy: 0.7789
Epoch 5/10
99000/99000 [==============================] - 204s 2ms/step - loss: 0.7760 - accuracy: 0.7818
Epoch 6/10
99000/99000 [==============================] - 205s 2ms/step - loss: 0.8007 - accuracy: 0.7837
Epoch 7/10
99000/99000 [==============================] - 204s 2ms/step - loss: 0.8007 - accuracy: 0.7851
Epoch 8/10
99000/99000 [==============================] - 202s 2ms/step - loss: 0.8224 - accuracy: 0.7861
Epoch 9/10
99000/99000 [==============================] - 206s 2ms/step - loss: 0.7750 - accuracy: 0.7869
Epoch 10/1

313/313 [==============================] - 1s 1ms/step
tf.Tensor(
[[5060  380  100   61   14   12    9    2    2    2    4    1    3    0
     0    3    1    0    0    0    0    0    0]
 [ 290 1248    4    0   28   20    7    1    6    0    0    2    2    0
     0    0    0    0    0    0    0    0    0]
 [  24    2  664    6    0    3    0    0    0    0    0    3    0    0
     0    0    0    0    0    0    0    0    0]
 [ 174   63   11  202    0   42    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0]
 [ 137   36    0    1   91    1    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0]
 [  63   31    5   27    1  138    0    6    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0]
 [  97   54    0    1    3    0   42    0    0    0    1    1    0    0
     0    0    0    0    0    0    0    0    0]
 [  32   10    2    0    0    2    0  131    0    0    0    0    0    0
     0    0    0    0 